In [ ]:
just_checking_integrity=False

In [ ]:
import gc
import pandas as pd
import scipy.sparse as sps
import numpy as np

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
URM_test=sps.load_npz("./drive/MyDrive/Recommendation_system/dataset/processed_data/URM_test_final.npz")

In [ ]:
URM_test.shape

(4439967, 23690)

In [ ]:
np.ravel(URM_test.sum(axis=1)).shape

(4439967,)

In [ ]:
Session_ids=np.argwhere(np.ravel(URM_test.sum(axis=1)))

In [ ]:
is_sorted = lambda a: np.all(a[:-1] <= a[1:])

In [ ]:
is_sorted(Session_ids)

True

In [ ]:
dataset=None
dataset_max=None
#["UCF","ICBF","ICF","Graph","TopPop100","TopPop50","TopPop15"] # "UCF_W","Graph_W","ICF_W","UCF_WW","ICBF_WW","Graph_WW","ICF_WW", "ICBF_W"
for file in c:
    print(gc.collect())
    print(f"loading {file}")
    df=pd.read_csv(f"./drive/MyDrive/Recommendation_system/dataset/candidates/traditional_recs/final/{file}.csv")
    print(len(df))
    print(f"loaded {file}")
    for col in df:
        if "Max" in col:
            df_max=df[["Session_Id",col]].drop_duplicates()
        elif "Score" in col:
            df_feats=df[["Session_Id","Item_ID",col]]
    if dataset is None:
        dataset=df_feats
        dataset_max=df_max
    else:
        dataset=dataset.merge(df_feats,on=["Session_Id","Item_ID"],how="outer")
        dataset_max=dataset_max.merge(df_max,on=["Session_Id"],how="outer")
    del df


del df_feats
del df_max

0
loading UCF
15960976
loaded UCF
0
loading ICBF
14855475
loaded ICBF
0
loading Graph
8321415
loaded Graph
0
loading ICF
17771428
loaded ICF


In [ ]:
ids=np.sort(dataset["Session_Id"].unique())
map_ids=pd.DataFrame()
map_ids["Session_Id"]=ids
if just_checking_integrity:
    map_ids["Real_Id"]=Session_ids[:len(ids)]
else:
    map_ids["Real_Id"]=Session_ids


In [ ]:
dataset_max=dataset_max.merge(map_ids,on="Session_Id",how="left")
del dataset_max["Session_Id"]
dataset_max.rename(columns={"Real_Id":"Session_Id"},inplace=True)

In [ ]:
dataset=dataset.merge(map_ids,on="Session_Id",how="left")
del dataset["Session_Id"]
dataset.rename(columns={"Real_Id":"Session_Id"},inplace=True)

In [ ]:
# import ast
# submission_df=pd.read_csv("./drive/MyDrive/Recommendation_system/dataset/candidates/NN/GRU/final/candidates.csv")
# submission_df["score"]=submission_df["score"].map(ast.literal_eval)
# submission_df["code"]=submission_df["code"].map(ast.literal_eval)
# submission_df["rank"]=submission_df["rank"].map(ast.literal_eval)
# submission_df = submission_df.explode(['code','score', 'rank'])
# submission_df[["session_id","code","rank"]]=submission_df[["session_id","code","rank"]].astype(int)
# submission_df.rename(columns={"code":"Item_ID","session_id":"Session_Id","rank":"rank_gru","score":"score_gru"},inplace=True)

In [ ]:
# import ast
# submission_df_tr=pd.read_csv("./drive/MyDrive/Recommendation_system/dataset/candidates/NN/Transformer/final/candidates.csv")
# submission_df_tr["score"]=submission_df_tr["score"].map(ast.literal_eval)
# submission_df_tr["code"]=submission_df_tr["code"].map(ast.literal_eval)
# submission_df_tr["rank"]=submission_df_tr["rank"].map(ast.literal_eval)
# submission_df_tr = submission_df_tr.explode(['code','score', 'rank'])
# submission_df_tr[["session_id","code","rank"]]=submission_df_tr[["session_id","code","rank"]].astype(int)
# submission_df_tr.rename(columns={"code":"Item_ID","session_id":"Session_Id","rank":"rank_transformer","score":"score_transformer"},inplace=True)

In [ ]:
import ast
submission_df_LSTM=pd.read_csv("./drive/MyDrive/Recommendation_system/dataset/candidates/NN/LSTM/final/candidates.csv")
submission_df_LSTM["score"]=submission_df_LSTM["score"].map(ast.literal_eval)
submission_df_LSTM["code"]=submission_df_LSTM["code"].map(ast.literal_eval)
submission_df_LSTM["rank"]=submission_df_LSTM["rank"].map(ast.literal_eval)
submission_df_LSTM = submission_df_LSTM.explode(['code','score', 'rank'])
submission_df_LSTM[["session_id","code","rank"]]=submission_df_LSTM[["session_id","code","rank"]].astype(int)
submission_df_LSTM.rename(columns={"code":"Item_ID","session_id":"Session_Id","rank":"rank_LSTM","score":"score_LSTM"},inplace=True)

In [ ]:
# dataset=dataset.merge(submission_df,on=["Session_Id","Item_ID"],how="outer")
# del submission_df
# dataset=dataset.merge(submission_df_tr,on=["Session_Id","Item_ID"],how="outer")
# del submission_df_tr
dataset=dataset.merge(submission_df_LSTM,on=["Session_Id","Item_ID"],how="outer")


In [ ]:
dataset=dataset.merge(dataset_max,on=["Session_Id"],how="left")
del dataset_max

In [ ]:
dataset.fillna(0,inplace=True)

In [ ]:
gc.collect()

2288

In [ ]:
dataset_macro_features = pd.read_csv('./drive/MyDrive/Recommendation_system/dataset/processed_data/macro_feats_NN_final.csv',
                                     usecols=['session_id',
                                                'session_similarity',
                                                'session_similarity_uniques', 'session_similarity_feats', 'session_similarity_feats_uniques',
                                              'length_of_session_seconds',
                   'avg_time_spent_per_item_seconds',
                   'variance_time_spent_per_item_seconds',
                   'n_seen_items',
                   'n_unique_items',])

dataset_macro_features.rename(columns={'session_id':'Session_Id'}, inplace=True)
dataset = dataset.merge(dataset_macro_features, on='Session_Id', how='left')

In [ ]:
# dataset.drop('Max_Score_TopPop15', axis=1, inplace=True)

In [ ]:
dataset.head()

,Item_ID,Score_UCF,Score_ICBF,Score_Graph,Score_ICF,Session_Id,score_LSTM,rank_LSTM,Max_Score_UCF,Max_Score_ICBF,...,Max_Score_ICF,length_of_session_seconds,avg_time_spent_per_item_seconds,variance_time_spent_per_item_seconds,n_seen_items,n_unique_items,session_similarity,session_similarity_uniques,session_similarity_feats,session_similarity_feats_uniques
0,11,0.035145,0.115033,0.006554,0.311263,61,0.001511,45.0,239.07347,3.357151,...,0.563566,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0
1,19,0.018681,0.000000,0.000000,0.000000,61,0.000000,0.0,239.07347,3.357151,...,0.563566,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0
2,23,0.039035,0.072503,0.007389,0.325638,61,0.000000,0.0,239.07347,3.357151,...,0.563566,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0
3,25,0.025481,0.081094,0.004554,0.006163,61,0.000000,0.0,239.07347,3.357151,...,0.563566,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0
4,69,0.056301,0.085104,0.011284,0.338065,61,0.002591,25.0,239.07347,3.357151,...,0.563566,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0


In [ ]:
gc.collect()

0

In [ ]:
dataset=dataset.sort_values("Session_Id")

In [ ]:
gc.collect()

0

In [ ]:
import lightgbm

In [ ]:
k=10
result = []

In [ ]:
for index in range(k):
    model = lightgbm.Booster(model_file=f'./drive/MyDrive/Recommendation_system/rankers/lightgbm/all_traditional_and_lstm/model_{index}.txt')
    result.append(model.predict(dataset.drop(["Session_Id","Item_ID"],axis=1)))

result = np.array(result)
result

array([[-0.32250742, -1.82630611, -1.89948004, ..., -1.86377808,
        -1.86377808, -1.53990971],
       [-0.86963227, -3.17537099, -3.31268778, ..., -3.19849513,
        -3.19849513, -2.36470167],
       [-1.08337799, -3.4424639 , -3.55231233, ..., -3.32397016,
        -3.32397016, -2.57951804],
       ...,
       [-1.18756523, -3.58511242, -3.78713499, ..., -3.4148288 ,
        -3.4148288 , -2.1746211 ],
       [-1.10793473, -3.4705367 , -3.75911079, ..., -3.36060508,
        -3.36060508, -2.51644264],
       [-1.21864832, -3.4654535 , -3.746795  , ..., -3.3785058 ,
        -3.3785058 , -2.5015894 ]])

In [ ]:
dataset["score"]=np.mean(result ,axis=0)
# dataset["score"]=np.median(result ,axis=0)

In [ ]:
dataset=dataset[["Session_Id","Item_ID","score"]]
gc.collect()

49

In [ ]:
dataset.head()

,Session_Id,Item_ID,score
0,61,11,-0.911340
25325335,61,2839,-3.005523
25325334,61,2721,-3.162443
25325333,61,2640,-3.162443
25325332,61,2594,-3.164590


In [ ]:
dataset=dataset.groupby(['Session_Id']).agg(list).reset_index()

In [ ]:
def get_indices(x):
    x=np.array(x)
    indices_unsorted=np.argpartition(x, -100)[-100:]
    x=x[indices_unsorted]
    indices=indices_unsorted[np.argsort(-x)]
    return indices

In [ ]:
dataset["indices"]=dataset["score"].apply(lambda x: get_indices(x))

In [ ]:
dataset.iloc[0]

Session_Id                                                   61
Item_ID       [11, 2839, 2721, 2640, 2594, 2442, 2386, 2376,...
score         [-0.9113398033877667, -3.005522745285509, -3.1...
indices       [386, 407, 537, 448, 75, 98, 531, 400, 296, 10...
Name: 0, dtype: object

In [ ]:
dataset.iloc[0]["Item_ID"][536]

3581

In [ ]:
def get_items(x):
    items=np.array(x[0])
    return items[x[1]]

In [ ]:
dataset["predictions"]=dataset[["Item_ID","indices"]].apply(get_items,axis=1)

In [ ]:
dataset=dataset[["Session_Id","predictions"]]
dataset["rank"]=dataset.apply(lambda x:tuple([rank for rank in range(1, 101)]) ,axis=1)

In [ ]:
dataset

,Session_Id,predictions,rank
0,61,"[1045, 292, 3586, 1563, 4574, 3904, 3414, 143,...","(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."
1,96,"[3564, 3705, 3778, 4591, 143, 3290, 4148, 1183...","(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."
2,185,"[4148, 4146, 1936, 2704, 2330, 3326, 3827, 416...","(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."
3,224,"[4202, 3338, 2769, 16, 4990, 4764, 1516, 2448,...","(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."
4,285,"[3691, 3555, 2899, 2970, 3441, 2471, 3599, 465...","(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."
...,...,...,...
49995,4439646,"[270, 1468, 772, 4026, 3656, 2720, 3424, 191, ...","(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."
49996,4439648,"[4145, 3901, 3223, 4586, 3550, 3110, 3505, 223...","(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."
49997,4439675,"[4865, 3166, 4966, 819, 779, 3445, 3879, 1169,...","(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."
49998,4439868,"[519, 4740, 1972, 2285, 292, 4220, 3414, 3586,...","(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."


In [ ]:
dataset["predictions"]=dataset["predictions"].apply(tuple)

In [ ]:
dataset=dataset[["Session_Id","predictions","rank"]]

In [ ]:
dataset.columns=["session_id","code","rank"]

In [ ]:
dataset.head()

,session_id,code,rank
0,61,"(1045, 292, 3586, 1563, 4574, 3904, 3414, 143,...","(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."
1,96,"(3564, 3705, 3778, 4591, 143, 3290, 4148, 1183...","(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."
2,185,"(4148, 4146, 1936, 2704, 2330, 3326, 3827, 416...","(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."
3,224,"(4202, 3338, 2769, 16, 4990, 4764, 1516, 2448,...","(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."
4,285,"(3691, 3555, 2899, 2970, 3441, 2471, 3599, 465...","(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."


In [ ]:
submission_df = dataset.explode(['code', 'rank']).astype(int)

In [ ]:
mapping= pd.read_csv('./drive/MyDrive/Recommendation_system/dataset/processed_data/map.csv')
submission_df=submission_df.merge(mapping,on="code",how="left")
del submission_df["code"]
submission_df=submission_df[["session_id","item_id","rank"]]

In [ ]:
print(len(submission_df))

5000000


In [ ]:
submission_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5000000 entries, 0 to 4999999
Data columns (total 3 columns):
 #   Column      Dtype
---  ------      -----
 0   session_id  int64
 1   item_id     int64
 2   rank        int64
dtypes: int64(3)
memory usage: 152.6 MB


In [ ]:
submission_df.isna().sum()

session_id    0
item_id       0
rank          0
dtype: int64

In [ ]:
submission_df.to_csv(
    f'./drive/MyDrive/Recommendation_system/rankers/lightgbm/all_traditional_and_lstm/final/final_sub.csv',
    index=False
    )